In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
# Import the loading function from xBOUT
from xbout import open_boutdataset

# Import xarray, which is the real hero here
import xarray as xr

### Loading data

This tutorial assumes you have already run the `delta_1` case of the `blob2d` [example](https://github.com/boutproject/BOUT-dev/tree/master/examples/blob2d) from the BOUT++ repository.

In [ ]:
# Open the data and see what's inside
ds = open_boutdataset('./delta_1/BOUT.dmp.*.nc', inputfilepath='./delta_1/BOUT.inp')

You can see all the variables, but none of them have been loaded yet, thanks to xarray's "lazy loading".

Each variable depends on dimensions or `dims`, which are like axes of numpy arrays but labelled with a name.

The `Dataset` is like a dictionary container of multiple `DataArrays`, each of which represents a simulation variable.

We have also stored various unphysical simulation quantities (such as processor splitting) in the `attrs` of the datasets, which is just a dictionary for carrying arbitrary extra info about the dataset's contents. 

Because we specified the path to the input file, the run options have also been included as a `ConfigParser` object.

Currently the dataset has no coordinates, as the warning mentions, but we will come back to that.

The object we've loaded is an `xarray.Dataset`, filled out in a sensible way for BOUT++ data. One way to think of it is as an in-memory representation of a netCDF file. Another way is like a set of numpy arrays with labelled axes. (If you've used pandas then it's also like a multidimensional pandas series.)

You will find the [xarray documentation](https://xarray.pydata.org/en/stable/index.html) useful.

In [ ]:
# You can see we have an unneccessary y dimension of length 1, so let's drop that
ds = ds.squeeze(drop=True)
ds

xarray is just wrapping numpy arrays, so we can always get our values back if we want

In [ ]:
ds['t_array'].values

### Basic plotting with xarray

In [ ]:
# What does the data look like? Let's try plotting a single frame
# Choose the density, at the 10th time index
plt.figure()
ds['n'].isel(t=10).plot()

## It's a blob!

Note xarray's `.plot()` method just wraps matplotlib, so you can pass matplotlib commands straight to it:

In [ ]:
plt.figure()
ds['n'].isel(t=10).plot(cmap='plasma')

In [ ]:
# Let's watch it fly
ds['n'].bout.animate2D()

(xarray plots do not open a new figure, so we need to create a new one first. The ds.bout plotting and animation methods do create a new figure.)

In [ ]:
# Now let's animate everything
ds.bout.animate_list(['n', 'phi', 'omega'], nrows=3)

Swirly.

### Coordinates

Let's add some coordinates for the radial and binormal directions.

In [ ]:
# dz is a scalar, so gets stored in 'metadata'
dz = xr.DataArray(ds.metadata['dz']).expand_dims({'z': ds.dims['z']})
z = dz.cumsum(dim='z')
ds = ds.assign_coords({'z': z})

# We already have dx, so let's use that
x = ds['dx'].cumsum(dim='x')
ds = ds.assign_coords({'x': x})

# The time array is also already in the data
ds = ds.assign_coords({'t': ds['t_array']})

# Now we have some coordinates!
ds

In [ ]:
# (Aside:) It would be nice to do all that every time we open our data, e.g.
def open_blob2ddataset(datapath='./BOUT.dmp.*', inputfilepath='./BOUT.inp'):
    ds = open_boutdataset(datapath=datapath, inputfilepath=inputfilepath).squeeze(drop=True)

    # add your physics-model-specific coordinates here

    return ds

Coordinates (or `coords`) are basically just data variables that have been given special status.

However, they can also be used to index the data.

### Indexing

In [ ]:
ds['n'].isel(t=10)  # selects the 10th slice along the t dimension

In [ ]:
ds['n'].sel(t=50)  # selects the slice which has a t value of 50

In [ ]:
# If you have scipy installed, you can also interpolate the data values
ds['n'].interp(t=55)

### Units

In [ ]:
# The metadata can be read for plotting
ds['t'].attrs['units'] = '1/wci'
ds['x'].attrs['units'] = 'rhos'
ds['z'].attrs['units'] = 'rhos'

In [ ]:
# (full integration of BoutOptionsFile and xBOUT will come...)
# note BOUT.settings stores all the options actually used in a run in input-file format
from boutdata.data import BoutOptionsFile
options = BoutOptionsFile('./delta_1/BOUT.settings')

# We can un-normalise our data, so it's in physical units
n0 = options['model']['n0']
ds['n'] = ds['n'] * n0
ds['n'].attrs['units'] = 'm-3'

In [ ]:
# Now if we plot again we will see the units too
plt.figure()
ds['n'].isel(t=10).plot()

### Now let's do some physics. 

Let's find the velocity of the centre-of-mass of just the filament.

In [ ]:
# Select just the filament, defined as the region with density greater than some multiple of the background
threshold = 1.1 * n0
blob = ds.where(ds['n'] > threshold)

# Now all the rest of the data has been replaced with NaNs
# xarray will exclude the NaNs when plotting
plt.figure()
blob['n'].isel(t=10).plot()

In [ ]:
# Calculate CoM
n, x = ds['n'], ds['x']

ntotal = n.sum(dim=['x','z'])
xCoM = (x*n).sum(dim=['x','z']) / ntotal
zCoM = (z*n).sum(dim=['x','z']) / ntotal

In [ ]:
# Find velocity of CoM
v_xCoM = xCoM.differentiate('t')
v_zCoM = zCoM.differentiate('t')

# This quantity is 1D - but xarray knows to use a line plot to plot it 
plt.figure()
v_xCoM.plot()

In [ ]:
# Contours of electric potential on top of vorticity
plt.figure()
ds['omega'].isel(t=15).plot()
ds['phi'].isel(t=15).plot.contour(center=0.0, cmap='seismic')

## xBOUT's calc module

`xBOUT` is also a good place to store analysis methods and functions which are likely to be useful to many BOUT++ users. These should go in the `calc` module.

# Advanced

### Analysing multiple runs together

In [ ]:
# We can load the results from multiple simulation runs into a single dataset.
# This is great for analysing parameter scans
widths = [0.25, 1, 10]
runs = []
for w in widths:
    run = open_blob2ddataset(datapath=f'./delta_{w}/BOUT.dmp.*.nc', inputfilepath=f'./delta_{w}/BOUT.inp')
    runs.append(run)
width_coord = xr.DataArray(amplitudes, dims='w')
scan = xr.concat(runs, dim=width_coord)
print(scan)

However, this isn't going to work with the standard blob2d example data.
That's because the default options for the three cases produce time series of different lengths.

You will have to re-run the simulations with the same time resolution and length if you want to combine them like this.

In [ ]:
# Calculate the velocities separately then join together for plotting?

In [ ]:
# Now we have all the runs together, we can compare them easily
# Plot of average velocities against amplitude?
# Compare to analytic scaling?

### Physics-model-specific accessors

`xbout` achieves the `ds.bout.method()` syntax by using the ["accessor"](https://xarray.pydata.org/en/stable/internals.html#extending-xarray) interface provided by xarray.

This is great because it allows us to attach domain specific functionality (i.e. tokamak-specific plotting methods) to general data structures (i.e. `xarray.Dataset` objects).

We can go further though.

The accessor classes `BoutDatasetAccessor` and `BoutDataArrayAccessor` are intended to be subclassed for specific BOUT++ modules. 
The subclass accessor will then inherit all the `.bout` accessor methods, but you will also be able to override these and define your own methods within your new accessor.

For example to add an extra method specific to the STORM BOUT++ module:

In [ ]:
from xarray import register_dataset_accessor
from xbout.boutdataset import BoutDatasetAccessor

@register_dataset_accessor('storm')
class StormAccessor(BoutDatasetAccessor):
    def __init__(self, ds_object):
        super().__init__(ds_object)

    def special_method(self):
        print("Do something only STORM users would want to do")

ds.storm.special_method()

### Example of using accessors

In [ ]:
from xarray import register_dataset_accessor

from xbout import BoutDatasetAccessor, BoutDataArrayAccessor


@register_dataset_accessor('utils')
class UtilityDatasetAccessor(BoutDatasetAccessor):
    """
    Class specifically for calculating ExB velocities of BOUT++ data.
    
    Requires that the BOUT++ data has a 'phi' field and 'x' and 'z' coordinates,
    in addition to the default 'Bxy' magnetic field.
    """

    def __init__(self, ds):
        super().__init__(ds)
        self.data = ds

    # This is where module-specific methods would go
    # For example maybe elm-pb would have a .elm_growth_rate() method?

    @property
    def v_radial(self):
        """Calculates local radial ExB velocity"""

        if 'v_radial' not in self.data:
            E_z = self.data['phi'].differentiate(coord='z')
            v_radial = E_z / self.data['Bxy']
            v_radial.attrs['standard_name'] = 'radial velocity'
            self.data['v_radial'] = v_radial
        return self.data['v_radial']

    @property
    def v_binormal(self):
        """Calculates local binormal ExB velocity"""

        if 'v_binormal' not in self.data:
            E_x = self.data['phi'].differentiate(coord='x')
            v_binormal = -E_x / self.data['Bxy']
            v_binormal.attrs['standard_name'] = 'binormal velocity'
            self.data['v_binormal'] = v_binormal
        return self.data['v_binormal']

We've written the methods in like this (using the property decorator) so that they can be calculated like variables which already exist, and then saved on the dataset.

In [ ]:
# Choose a time slice, downsample, and select region of interest
bd = ds.isel(t=10, x=slice(50, 150, 3), z=slice(75, 175, 3))

# Find ExB velocities
vx = bd.utils.v_radial
vz = bd.utils.v_binormal

bd

In [ ]:
# Plot streamlines
fig, ax = plt.subplots()
bd['phi'].transpose().plot.contour(center=0.0, cmap='seismic', ax=ax)

# Plot the flow
# (we're using pure matplotlib so have to match up the dimensions)
x, z = bd['x'].broadcast_like(vx), bd['z'].broadcast_like(vx)
ax.quiver(x.values, z.values, vx.values, vz.values, scale=1.7)

### Your own data

If you have some of your own data, try loading that.
For 3D tokamak datasets specify `geometry='toroidal'` in `open_boutdataset`.
Then when plotting try `ds[var].bout.pcolormesh()`